<a href="https://colab.research.google.com/github/raulmartosmoreno/FOOTAGEP-ESTABLE-V6/blob/main/200_caras_ia.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# @title GENERADOR V5: CARAS IA (Efecto Morphing Real)
# @markdown ---
# @markdown **ESTRATEGIA:**
# @markdown 1. Descargamos caras generadas por IA (siempre alineadas).
# @markdown 2. Al tener los ojos en el mismo sitio, el ordenamiento crea una ilusión de movimiento fluida.
# @markdown 3. **Nota:** Bajaremos 200 caras para la prueba (tarda un poco más porque hay que ser educado con el servidor).
# @markdown ---

import os
import requests
import cv2
import numpy as np
import time
from sklearn.manifold import TSNE
from google.colab import files
import shutil

# --- CONFIGURACIÓN ---
NUM_IMAGENES = 200 # @param {type:"integer"}
RESOLUCION = 1080
TEMP_FOLDER = "caras_ia_v5"
FPS_ARCHIVO = 30
FRAMES_POR_IMAGEN = 2 # Ritmo visual = 15 fps

# --- FUNCIONES ---

def download_ai_faces(num_images, folder):
    print(f"⬇️ Hackeando {num_images} identidades falsas (IA)...")
    if os.path.exists(folder):
        shutil.rmtree(folder)
    os.makedirs(folder)

    # Usamos headers para parecer un navegador humano y no un bot
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
    }

    session = requests.Session()

    for i in range(num_images):
        # Esta URL genera una cara nueva cada vez que la llamas
        url = "https://thispersondoesnotexist.com"

        try:
            response = session.get(url, headers=headers, timeout=10)
            if response.status_code == 200:
                file_path = os.path.join(folder, f"face_{i:03d}.jpg")
                with open(file_path, 'wb') as f:
                    f.write(response.content)

                # IMPORTANTE: Esperamos un poco entre fotos para no saturar
                # y que nos den caras distintas
                time.sleep(0.8)

            if (i + 1) % 10 == 0:
                print(f"   [Generando] {i + 1}/{num_images} caras...")
        except Exception as e:
            print(f"   ⚠️ Fallo en cara {i}: {e}")

    print("✅ Base de datos de caras completada.\n")

def get_structural_features(img):
    # Pasamos a gris y reducimos mucho para que la IA se fije
    # en la iluminación y la posición de la cabeza, no en los detalles.
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    small_gray = cv2.resize(gray, (64, 64), interpolation=cv2.INTER_AREA)
    return small_gray.flatten()

def sort_images_by_structure(folder_path):
    print("💀 Analizando fisonomía (t-SNE)...")
    images_features = []
    filenames = []

    valid_files = sorted([os.path.join(folder_path, f) for f in os.listdir(folder_path) if f.endswith('.jpg')])

    for file_path in valid_files:
        img = cv2.imread(file_path)
        if img is not None:
            features = get_structural_features(img)
            images_features.append(features)
            filenames.append(file_path)

    if not images_features: return []

    print("🧬 Ordenando secuencia genética...")
    X = np.array(images_features)
    # Ajustamos parámetros para caras
    tsne = TSNE(n_components=1, perplexity=30, n_iter=1000, random_state=42, init='pca', learning_rate='auto')
    X_embedded = tsne.fit_transform(X)

    sorted_indices = np.argsort(X_embedded[:, 0])
    return [filenames[i] for i in sorted_indices]

def create_video(sorted_files, output_file='video_caras_morph.mp4'):
    print(f"🎬 Renderizando pesadilla visual a 30fps...")

    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    out = cv2.VideoWriter(output_file, fourcc, FPS_ARCHIVO, (RESOLUCION, RESOLUCION))

    for i, file_path in enumerate(sorted_files):
        img = cv2.imread(file_path)
        if img is not None:
            # Crop inteligente (las caras de esta IA suelen estar centradas)
            h, w, _ = img.shape
            min_dim = min(h, w)
            start_x = (w - min_dim) // 2
            start_y = (h - min_dim) // 2
            img_crop = img[start_y:start_y+min_dim, start_x:start_x+min_dim]
            img_final = cv2.resize(img_crop, (RESOLUCION, RESOLUCION), interpolation=cv2.INTER_LINEAR)

            for _ in range(FRAMES_POR_IMAGEN):
                out.write(img_final)

    out.release()
    print(f"🎉 ¡VÍDEO DE CARAS LISTO! {output_file}")
    files.download(output_file)

# --- EJECUCIÓN ---
download_ai_faces(NUM_IMAGENES, TEMP_FOLDER)
sorted_list = sort_images_by_structure(TEMP_FOLDER)
create_video(sorted_list)

⬇️ Hackeando 200 identidades falsas (IA)...
   [Generando] 10/200 caras...
   [Generando] 20/200 caras...
   [Generando] 30/200 caras...
   [Generando] 40/200 caras...
   [Generando] 50/200 caras...
   [Generando] 60/200 caras...
   [Generando] 70/200 caras...
   [Generando] 80/200 caras...
   [Generando] 90/200 caras...
   [Generando] 100/200 caras...
   [Generando] 110/200 caras...
   [Generando] 120/200 caras...
   [Generando] 130/200 caras...
   [Generando] 140/200 caras...
   [Generando] 150/200 caras...
   [Generando] 160/200 caras...
   [Generando] 170/200 caras...
   [Generando] 180/200 caras...
   [Generando] 190/200 caras...
   [Generando] 200/200 caras...
✅ Base de datos de caras completada.

💀 Analizando fisonomía (t-SNE)...
🧬 Ordenando secuencia genética...


/usr/local/lib/python3.12/dist-packages/sklearn/manifold/_t_sne.py:1164: FutureWarning: 'n_iter' was renamed to 'max_iter' in version 1.5 and will be removed in 1.7.
  warnings.warn(


🎬 Renderizando pesadilla visual a 30fps...
🎉 ¡VÍDEO DE CARAS LISTO! video_caras_morph.mp4


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>